In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('data/sample201320.csv', low_memory=False)

In [4]:
df.shape

(193310, 127)

In [5]:
df.head(2)

,RecordID,RECORD_NUM,VIR_CERT_NUMBER,STATION_NUM,ANALYZER_NUM,INSPECTOR_NUM,TEST_TYPE,VID_TEST_TYPE,TEST_SDATE,TEST_EDATE,VIN,VIN_SOURCE,VEH_LICENSE,GVW_TYPE,GVWR,MODEL_YEAR,MAKE,MODEL,CYL,ENGINE_SIZE,TRANS_TYPE,DUAL_EXHAUST,ODOMETER,FUEL_TYPE,VEHICLE_TYPE,EMISS_COST,ZIP_CODE,CERTIFICATION,ESC,VRT_RECORD_ID,E_TEST_SEQUENCE,E_PRECOND_PROCEDURE,E_HIGH_DCF,E_HIGH_RPM,E_HIGH_CO2,E_HIGH_O2,E_HIGH_HC,E_HIGH_HC_DCF,E_HIGH_HC_LIMIT,E_HIGH_CO,E_HIGH_CO_DCF,E_HIGH_CO_LIMIT,E_IDLE_DCF,E_IDLE_RPM,E_IDLE_CO2,E_IDLE_O2,E_IDLE_HC,E_IDLE_HC_DCF,E_IDLE_HC_LIMIT,E_IDLE_CO,E_IDLE_CO_DCF,E_IDLE_CO_LIMIT,E_HIGH_DCF_2,E_HIGH_RPM_2,E_HIGH_CO2_2,E_HIGH_O2_2,E_HIGH_HC_2,E_HIGH_HC_DCF_2,E_HIGH_CO_2,E_HIGH_CO_DCF_2,E_IDLE_DCF_2,E_IDLE_RPM_2,E_IDLE_CO2_2,E_IDLE_O2_2,E_IDLE_HC_2,E_IDLE_HC_DCF_2,E_IDLE_CO_2,E_IDLE_CO_DCF_2,E_RESULT_STRING,V_SMOKE1,V_SMOKE2,V_GASCAP,V_CAT,V_RESULT,KOEO_RESULT,KOER_RESULT,OBD_RDY_MISFIRE,OBD_RDY_FUEL,OBD_RDY_COMPONENT,OBD_RDY_CAT,OBD_RDY_CAT_HEATED,OBD_RDY_EVAP,OBD_RDY_SEC_AIR,OBD_RDY_AIR_COND,OBD_RDY_O2_SENSOR,OBD_RDY_O2_HEATER,OBD_RDY_EGR,OBD_RDY_RESULT,OBD_PID,OBD_PCM_ID,OBD_VIN,OBD_FLT_CODES,OBD_FLT_RESULT,OBD_MIL_STATUS,OBD_RESULT,ONLINE_STATUS,OFFLINE_REASON_CODE,ABORT_CODE,SOFTWARE_VERSION,INSP_COMMENTS,OVERALL_RESULT,R_TOTAL_COST,R_EGR,R_ELECTRICAL,R_EVAP_CONTROL,R_EXHAUST,R_IGNITION,R_INTAKE,R_ENGINE,R_PCV,R_OTHER,SentToWW,ABORT_CODE_OTHER_DESC,OBDII_AFFECT_OVERALL,E_HIGH_CO_RESULT,E_HIGH_HC_RESULT,E_IDLE_CO_RESULT,E_IDLE_HC_RESULT,MECHANIC_LAST_NAME,MECHANIC_FIRST_NAME,MECHANIC_MIDDLE_INIT,ENGINE_SIZE_ORIG,FAILED_IDLE_RPM,SENTTOVID,Station_License_Number,Analyzer_Number,Insert_DateTime
0,2690332,10433,6878785.0,IS311,W028,AC084149,A,A,2013-03-04 10:11:00,2013-03-04 10:14:00,1G4HP52K45U160372,S,368RBP,1.0,3875,2005,BUICK,LESABRE,6,3800,A,N,114034,G,1,5.0,87121,C,6.0,20050189,1,1,0.0,0,0.0,0.0,0,0,100,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0,100,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,N,N,P,P,P,P,P,P,1.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,P,19.0,10,1G4HP52K45U160372,NaN,P,1.0,P,0,0,NaN,502,NaN,P,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,0,NaN,N,N,N,N,N,Huber,Bryan,S,NaN,NaN,0,IS311,W028,2013-03-04 16:26:26
1,2804644,18225,6979071.0,IR317,W039,37826481,I,I,2013-07-22 16:55:00,2013-07-22 16:56:00,3VWSF71K66M706988,S,843NWS,1.0,3250,2006,VOLKSWAGEN,JETTA,5,2500,M,N,76338,G,1,16.0,87108,C,6.0,0,1,1,0.0,0,0.0,0.0,0,0,100,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0,100,0.0,0.0,1.0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,N,N,P,N,P,P,P,P,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,P,38.0,E8,3VWSF71K66M706988,NaN,P,1.0,P,1,0,NaN,502,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,N,N,N,N,N,NGUYEN,HANH,A,2.5L,NaN,0,IR317,W039,2013-07-22 17:41:06


In [20]:
cols = list(df.columns)
print(cols[:30])
print(cols[30:69])
print(cols[69:76])
print(cols[76:95])
print(cols[95:])

['RecordID', 'RECORD_NUM', 'VIR_CERT_NUMBER', 'STATION_NUM', 'ANALYZER_NUM', 'INSPECTOR_NUM', 'TEST_TYPE', 'VID_TEST_TYPE', 'TEST_SDATE', 'TEST_EDATE', 'VIN', 'VIN_SOURCE', 'VEH_LICENSE', 'GVW_TYPE', 'GVWR', 'MODEL_YEAR', 'MAKE', 'MODEL', 'CYL', 'ENGINE_SIZE', 'TRANS_TYPE', 'DUAL_EXHAUST', 'ODOMETER', 'FUEL_TYPE', 'VEHICLE_TYPE', 'EMISS_COST', 'ZIP_CODE', 'CERTIFICATION', 'ESC', 'VRT_RECORD_ID']
['E_TEST_SEQUENCE', 'E_PRECOND_PROCEDURE', 'E_HIGH_DCF', 'E_HIGH_RPM', 'E_HIGH_CO2', 'E_HIGH_O2', 'E_HIGH_HC', 'E_HIGH_HC_DCF', 'E_HIGH_HC_LIMIT', 'E_HIGH_CO', 'E_HIGH_CO_DCF', 'E_HIGH_CO_LIMIT', 'E_IDLE_DCF', 'E_IDLE_RPM', 'E_IDLE_CO2', 'E_IDLE_O2', 'E_IDLE_HC', 'E_IDLE_HC_DCF', 'E_IDLE_HC_LIMIT', 'E_IDLE_CO', 'E_IDLE_CO_DCF', 'E_IDLE_CO_LIMIT', 'E_HIGH_DCF_2', 'E_HIGH_RPM_2', 'E_HIGH_CO2_2', 'E_HIGH_O2_2', 'E_HIGH_HC_2', 'E_HIGH_HC_DCF_2', 'E_HIGH_CO_2', 'E_HIGH_CO_DCF_2', 'E_IDLE_DCF_2', 'E_IDLE_RPM_2', 'E_IDLE_CO2_2', 'E_IDLE_O2_2', 'E_IDLE_HC_2', 'E_IDLE_HC_DCF_2', 'E_IDLE_CO_2', 'E_IDLE_C